# This serves as a prototype for calling, cleaning, and aggregating data from Riot's API.
### My API key is private, but you can easily create a .env for yourself and store it here.

In [13]:
# Import Dependencies
from dotenv import load_dotenv
import os
import json
import pandas as pd
import requests
import urllib.parse
from pprint import pprint

# Load environment variables from the .env file
load_dotenv()

# Retrieve the Riot API key
riot_api_key = os.getenv("RIOT_API_KEY")

# Riot API base URL
base_url = "https://americas.api.riotgames.com"

In [2]:
# Grab a Summoner's puuid

def get_puuid(game_name, tag_line):
    """
    Get the PUUID (Player Unique ID) for a given summoner using their Riot ID (gameName and tagLine).
    
    Parameters:
    - game_name (str): Summoner's game name (e.g., 'SummonerOne')
    - tag_line (str): Summoner's tag line (e.g., '0xZZk')
    
    Returns:
    - str: The PUUID of the summoner, or None if the request fails.
    """

    # Endpoint to get account info by Riot ID
    endpoint = f"/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    
    # Construct the full URL
    url = base_url + endpoint
    
    # Define headers, including the API Key
    headers = {
        "X-Riot-Token": riot_api_key
    }
    
    # Make the API request
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract PUUID from the response
        return data.get("puuid", None)
    else:
        print(f"Error fetching PUUID: {response.status_code} - {response.text}")
        return None

In [3]:
# Testing my summoner name
unearthed_puuid = get_puuid('Unearthed','0xZZk')

In [4]:
# Grab Summoners last n matches

def get_match_ids(puuid, count=10):
    headers = {
        "X-Riot-Token": riot_api_key
    }
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": 0,
        "count": count
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()  # List of match IDs
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [5]:
# My last ten matches
match_ids = get_match_ids(unearthed_puuid,10)
match_ids

['NA1_5281512012',
 'NA1_5281473224',
 'NA1_5281439643',
 'NA1_5281403573',
 'NA1_5281237422',
 'NA1_5281223933',
 'NA1_5281158999',
 'NA1_5281147066',
 'NA1_5280865238',
 'NA1_5280824575']

In [6]:
# Hard coding this so I don't have to run the API again and get different numbers
match_ids = ['NA1_5278153946',
 'NA1_5278129671',
 'NA1_5278107868',
 'NA1_5278022110',
 'NA1_5277407560',
 'NA1_5277181456',
 'NA1_5277090894',
 'NA1_5277075378',
 'NA1_5277033937',
 'NA1_5277027770']

In [7]:
#Get match timeline data by match id.

def get_timeline_data(matchId):
    
    endpoint = f"/lol/match/v5/matches/{matchId}/timeline"


    headers = {
        "X-Riot-Token": riot_api_key
    }

    url = base_url + endpoint


    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()  # Timeline data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [9]:
#Test this and give me a JSON

test_match = match_ids[0]
match_json = get_timeline_data(test_match)
pprint(get_timeline_data(test_match))

{'info': {'endOfGameResult': 'GameComplete',
          'frameInterval': 60000,
          'frames': [{'events': [{'realTimestamp': 1746078730477,
                                  'timestamp': 0,
                                  'type': 'PAUSE_END'},
                                 {'itemId': 3865,
                                  'participantId': 0,
                                  'timestamp': 0,
                                  'type': 'ITEM_PURCHASED'},
                                 {'itemId': 3865,
                                  'participantId': 0,
                                  'timestamp': 0,
                                  'type': 'ITEM_PURCHASED'}],
                      'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
                                                                    'abilityPower': 0,
                                                                    'armor': 36,
                                                               

In [14]:
#Send this to a JSON to communicate with ChatGPT.

with open("data/prototype-data/test_data.json", "w") as f:
        json.dump(match_json, f, indent=4)
print("Saved successfully.")

Saved successfully.
